# Gensim, word2vec

Gensim - вообще библиотека для тематического моделирования текстов. Один из компонентов в ней - питоновская обёртка для word2vec (который в оригинале был написан на C++).

Если gensim у вас не стоит, то ставим:

`pip install gensim`

Поскольку иногда тренировка модели занимает много времени, то можно ещё вести лог событий.

In [1]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Процесс тренировки модели 

На вход модели даем текстовый файл, каждое предложение на отдельной строчке.

In [2]:
f = 'text.txt'
data = gensim.models.word2vec.LineSentence(f)

Инициализируем модель. Параметры в скобочках:
* data - данные, 
* size - размер вектора, 
* window - размер окна наблюдения,
* min_count - мин. частотность слова в корпусе, которое мы берем,
* sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

In [3]:
model = gensim.models.Word2Vec(data, size=500, window=10, min_count=2, sg=0)

2017-05-30 00:13:36,174 : INFO : collecting all words and their counts
2017-05-30 00:13:36,178 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-30 00:13:36,304 : INFO : collected 30172 word types from a corpus of 68570 raw words and 5432 sentences
2017-05-30 00:13:36,307 : INFO : Loading a fresh vocabulary
2017-05-30 00:13:36,408 : INFO : min_count=2 retains 6793 unique words (22% of original 30172, drops 23379)
2017-05-30 00:13:36,409 : INFO : min_count=2 leaves 45191 word corpus (65% of original 68570, drops 23379)
2017-05-30 00:13:36,467 : INFO : deleting the raw counts dictionary of 30172 items
2017-05-30 00:13:36,471 : INFO : sample=0.001 downsamples 34 most-common words
2017-05-30 00:13:36,474 : INFO : downsampling leaves estimated 37321 word corpus (82.6% of prior 45191)
2017-05-30 00:13:36,478 : INFO : estimated required memory for 6793 words and 500 dimensions: 30568500 bytes
2017-05-30 00:13:36,535 : INFO : resetting layer weights
2017-05-30 

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать.

In [4]:
model.init_sims(replace=True)

2017-05-30 00:13:40,834 : INFO : precomputing L2-norms of word weight vectors


Смотрим, сколько в модели слов:

In [6]:
print(len(model.wv.vocab))

6793


И сохраняем!

In [7]:
model.save('my.model')

2017-05-30 00:15:07,615 : INFO : saving Word2Vec object under my.model, separately None
2017-05-30 00:15:07,621 : INFO : not storing attribute syn0norm
2017-05-30 00:15:07,623 : INFO : not storing attribute cum_table
2017-05-30 00:15:08,144 : INFO : saved my.model


## Работа с моделью

Для каких-то своих индивидуальных нужд и экспериментов бывает полезно самому натренировать модель на нужных данных и с нужными параметрами. Но для каких-то общих целей модели уже есть как для русского языка, так и для английского.

Модели для русского скачать можно здесь - http://rusvectores.org/ru/models

Скачаем модель для русского языка, созданную на основе НКРЯ. Поскольку модели бывают разных форматов, то для них написаны разные функции загрузки; бывает полезно учитывать это в своем скрипте:

In [8]:
m = 'ruscorpora_1_300_10.bin.gz'
if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.KeyedVectors.load(m)

2017-05-30 00:15:15,307 : INFO : loading projection weights from ruscorpora_1_300_10.bin.gz
2017-05-30 00:15:35,994 : INFO : loaded (184973, 300) matrix from ruscorpora_1_300_10.bin.gz


In [9]:
model.init_sims(replace=True)

2017-05-30 00:15:40,106 : INFO : precomputing L2-norms of word weight vectors


Скажем, нам интересны такие слова (пример для русского языка):

In [10]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных).

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [11]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

день_NOUN
[ 0.02765099 -0.06994063  0.02292976  0.00478499 -0.0373687   0.06778284
  0.01259792 -0.09310838  0.10806882 -0.01369707]
неделя_NOUN 0.7186020016670227
утро_NOUN 0.6839577555656433
месяц_NOUN 0.6676345467567444
вечер_NOUN 0.6635110378265381
час_NOUN 0.619712233543396
полдень_NOUN 0.6096612215042114
суббота_NOUN 0.6011918783187866
ночь_NOUN 0.5900086164474487
накануне_ADV 0.5895365476608276
понедельник_NOUN 0.5813597440719604


ночь_NOUN
[ 0.00090496 -0.09171917  0.06209332  0.00910732 -0.03711092  0.0487812
  0.05916326 -0.12525907  0.02495096 -0.07734441]
вечер_NOUN 0.7375167608261108
рассвет_NOUN 0.7365224361419678
утро_NOUN 0.6998549103736877
полночь_NOUN 0.6829675436019897
напролет_ADV 0.6194927096366882
сумерки_NOUN 0.6130234003067017
ночной_ADJ 0.6043827533721924
бессонный_ADJ 0.5970311760902405
спать_VERB 0.5932828187942505
полдень_NOUN 0.5922213196754456


человек_NOUN
[-0.13945162  0.00953103  0.09669054 -0.0521334  -0.03127262  0.05055281
 -0.02525845  0.01031411 

Находим косинусную близость пары слов:

In [12]:
print(model.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.330222839481


Найди лишнее!

In [13]:
print(model.doesnt_match('яблоко_NOUN груша_NOUN виноград_NOUN банан_NOUN лимон_NOUN картофель_NOUN'.split()))

картофель_NOUN


Реши пропорцию!

In [14]:
print(model.most_similar(positive=['пицца_NOUN', 'россия_NOUN'], negative=['италия_NOUN'])[0][0])

пельмень_NOUN


Это конечно все хорошо, но как понять, какая модель лучше? Или вот например я сделал свою модель, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Их два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. Подробнее читаем тут:  
* http://www.aclweb.org/aclwiki/index.php?title=Google_analogy_test_set_(State_of_the_art)
* https://www.aclweb.org/aclwiki/index.php?title=SimLex-999_(State_of_the_art)

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [ ]:
model.accuracy('ruanalogy_upos.txt', restrict_vocab=3000000)